In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as spf

In [2]:
spark = SparkSession.builder.appName("anokhin").getOrCreate()

In [3]:
spark

## User-based Collaborative Filtering

### Собираем данные на Spark и сохраняем как pandas dataframe

In [4]:
spark.read.json("/user/anokhin/seminar-2/*/data.json").show(3, False)

+-----------+---------------------+-------+--------------+----+-------------+-----+----+
|experiments|latency              |message|recommendation|time|timestamp    |track|user|
+-----------+---------------------+-------+--------------+----+-------------+-----+----+
|[T1]       |0.01097559928894043  |next   |47055         |1.0 |1727289551284|47055|4298|
|[C]        |0.0016002655029296875|next   |48100         |0.64|1727289551292|45208|4670|
|[C]        |0.0013813972473144531|next   |39136         |0.0 |1727289551310|39136|786 |
+-----------+---------------------+-------+--------------+----+-------------+-----+----+
only showing top 3 rows



In [6]:
data = (
    spark.read.json("/user/anokhin/seminar-2/*/data.json")
        .select(spf.col("user"), spf.col("time"), spf.col("track"))
        .filter(spf.col("track").isNotNull())
        .groupBy(["user", "track"])
        .agg(
            spf.avg("time").alias("time")
        )
).toPandas()

data.head()

user  track  time
0  2563  40113  0.09
1  3908  15729  0.01
2  2116  34583  0.00
3  6916   5410  0.00
4   953   1738  0.05

In [7]:
data.to_json("/home/anokhin/week03.json", orient="records", lines=True)

### Готовим рекомендации для каждого пользователя

Рекомендовать пользователю треки, которые понравились похожим на него пользователям

$$\hat r_{ui} = h^{-1} \left( \frac{\sum_{v \in N_i(u)} w_{uv} h(r_{vi})}{\sum_{v \in N_i(u)} w_{uv}} \right)$$

$N_i(u)$ - соседи пользователя $u$, которые оценили айтем $i$,
$w_{uv}, w_{ij}$ - веса соседей, 
$h$ - функция нормализации



**Нормализация**: Номализуем так, чтобы вектор пользователя имел единичную длину

**Веса**: Похожих пользователей будем искать по *cosine similarity*

**Соседи**: в качестве соседей будем рассматривать всех пользователей. Q: Как это упростит формулу?

In [1]:
import tqdm
import json

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

import matplotlib.pyplot as pl
import seaborn as sns

In [2]:
data = pd.read_json("/Users/n.anokhin/Desktop/week03.json", lines=True)

data.head()

user  track  time
0  4743  42920  1.00
1  8448  48653  0.01
2  5505   8369  0.00
3  1617  22821  0.05
4  4764  33723  0.63

In [3]:
data["normalized_time"] = data.groupby("user")["time"].transform(lambda time: time / np.sqrt(np.sum(time * time)))

data.head()

user  track  time  normalized_time
0  4743  42920  1.00         0.301639
1  8448  48653  0.01         0.003725
2  5505   8369  0.00         0.000000
3  1617  22821  0.05         0.011289
4  4764  33723  0.63         0.107738

In [4]:
interactions = pd.pivot_table(data, values="normalized_time", index="user", columns="track").fillna(0)

print("Interactions matrix: \nshape=" + str(interactions.shape))
print("Sparsity=" + str((interactions != 0).values.sum() / interactions.size))

interactions.head()

Interactions matrix: 
shape=(10000, 49980)
Sparsity=0.0006736814725890356


track  0      1      2      3      4      5      6      7      8      9      \
user                                                                          
0        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

track  ...  49990  49991  49992  49993  49994  49995  49996  49997  49998  \
user   ...                                                                  
0      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

track  49999  
user          
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  

[5 rows x 49980 columns]

In [5]:
similarity_matrix = cosine_similarity(interactions) #Q: Размер?
np.fill_diagonal(similarity_matrix, 0) #Q: Зачем?

print("Mean positive neighbours per user=" + str((similarity_matrix > 0).sum(axis=1).mean()))
print("Mean negative neighbours per user=" + str((similarity_matrix < 0).sum(axis=1).mean()))

Mean positive neighbours per user=236.2318
Mean negative neighbours per user=0.0


In [7]:
user = 1337

figure, ax = pl.subplots()
sns.histplot(similarity_matrix[user][similarity_matrix[user] > 0], bins=20, ax=ax)

ax.set_title(f"Nonzero elements: " + str((similarity_matrix[user] > 0).sum()))

pass

In [8]:
# TODO: expected size: observed users x observed tracks
scores_matrix = np.matmul(similarity_matrix, interactions.values)

scores = pd.DataFrame(
    scores_matrix,
    index=interactions.index,
    columns=interactions.columns
)

scores[[0, 1, 2, 3, 4, 5]].head()

track         1    2         3         4         5
user                                              
0      0.014666  0.0  0.006857  0.000973  0.000000
1      0.000000  0.0  0.000000  0.000000  0.000000
2      0.003316  0.0  0.000000  0.000000  0.024739
3      0.000000  0.0  0.000000  0.000000  0.000000
4      0.000000  0.0  0.000000  0.000000  0.000278

In [9]:
scores[[0, 1, 2, 3, 4, 5]].head()

track    0         1    2         3         4         5
user                                                   
0      0.0  0.014666  0.0  0.006857  0.000973  0.000000
1      0.0  0.000000  0.0  0.000000  0.000000  0.000000
2      0.0  0.003316  0.0  0.000000  0.000000  0.024739
3      0.0  0.000000  0.0  0.000000  0.000000  0.000000
4      0.0  0.000000  0.0  0.000000  0.000000  0.000278

#### Глянем на рекомендации

In [10]:
BOTIFY_DATA_DIR = "/Users/n.anokhin/Projects/recsys-course/botify/data/"

In [11]:
tracks = pd.read_json(BOTIFY_DATA_DIR + "tracks.json", lines=True).set_index("track")
tracks.head()

artist                           title     genre    pop
track                                                                         
7                    Harmonia                   Sehr kosmisch  Pop_Rock  65688
0                       Björk                            Undo      None  57660
2               Dwight Yoakam                  You're The One   Country  55035
1      Florence + The Machine  Dog Days Are Over (Radio Edit)      None  52773
15              Kings Of Leon                         Revelry  Pop_Rock  48290

In [12]:
user = np.random.choice(scores.index)
k = 10

# data[data["user"] == user].sort_values("time").tail(10)

In [29]:
user_scores = pd.merge(
    scores.loc[user].sort_values(ascending=False)[:k].to_frame("score"),
    tracks, 
    left_index=True, 
    right_index=True,
    how="inner"
)

user_scores.loc[diff]

score         artist                              title genre  pop
track                                                                       
22057  0.056512      Goldfrapp                  It's Not Over Yet  None  156
114    0.056512       Hot Chip                       We Have Love  None  433
24470  0.056512  Laura Marling  Goodbye England (Covered In Snow)  Folk  485
32827  0.056512    Tom T. Hall                        I Like Beer  None  123
14718  0.056512   The Thermals              Returning to the Fold  None  147

In [30]:
user_interactions = pd.merge(
    interactions.loc[user].sort_values(ascending=False).to_frame("time"),
    tracks, 
    left_index=True, 
    right_index=True, 
    how="inner"
)

user_interactions[user_interactions["time"] != 0]

time                  artist                            title  \
track                                                                      
4299   0.348153                Blue Six  Music & Wine (Th'Attaboy Vocal)   
45291  0.348153                     Ayo                       These Days   
41690  0.348153  Medeski_ Martin & Wood                        Queen Bee   
11811  0.348153   Incredible Bongo Band                           Apache   
32304  0.348153        John Butler Trio         Daniella (Album Version)   
21552  0.348153         Ricchi E Poveri                   Se Mi Innamoro   
47224  0.348153             The Subways                      No Goodbyes   
11165  0.348153    Blood_ Sweat & Tears                   Spinning Wheel   
29955  0.170595                 Turisas             Five Hundred And One   
14898  0.034815          Ben Folds Five                             Mess   

          genre  pop  
track                 
4299       None  140  
45291  Pop_Rock   73  
41690      None  110  
11811  Pop_Rock  118  
32304      None  308  
21552      None   63  
47224  Pop_Rock   75  
11165      None  345  
29955  Pop_Rock   81  
14898  Pop_Rock   60

In [28]:
diff = set(recommended_tracks).difference(interaction_tracks)

## Подготавливаем рекомендации для продакшена

In [31]:
def recommend(user_id, scores, k):
    return scores.loc[user_id].sort_values(ascending=False)[:k].index.tolist()

In [32]:
users = data["user"].unique()

with open(BOTIFY_DATA_DIR + "recommendations_ub.json", "w") as rf:
    for user in tqdm.tqdm(users):
        recommendation = {
            "user": int(user),
            "tracks": recommend(user, scores, 100)
        }
        rf.write(json.dumps(recommendation) + "\n")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:24<00:00, 411.97it/s]
